In [1]:
from time import time
from datetime import timedelta
from copy import deepcopy

import random
import numpy as np
import pandas as pd
from ml_metrics import mapk

import torch
from torch.optim import AdamW
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForMultipleChoice

/home/teddy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/teddy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/teddy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/teddy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

In [2]:
# Random seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
# CUDA device
use_cuda_device = 0
torch.cuda.set_device(use_cuda_device)
print("Using CUDA device: %d" % torch.cuda.current_device())

Using CUDA device: 0


# Setting

In [3]:
# Input files
document_csv_path = './documents.csv'
training_csv_path = './train_queries.csv'
testing_csv_path = './test_queries.csv'

# Input limitation
max_query_length = 64
max_input_length = 512
num_negatives = 3   # num. of negative documents to pair with a positive document

# Model finetuning
model_name_or_path = "bert-base-uncased"
max_epochs = 1
learning_rate = 3e-5
dev_set_ratio = 0.2   # make a ratio of training set as development set for rescoring weight sniffing
max_patience = 0      # earlystop if avg. loss on development set doesn't decrease for num. of epochs
batch_size = 2    # num. of inputs = 8 requires ~9200 MB VRAM (num. of inputs = batch_size * (num_negatives + 1))
num_workers = 2   # num. of jobs for pytorch dataloader

# Save paths
save_model_path = "models/bert_base_uncased"  # assign `None` for not saving the model
save_submission_path = "bm25_bert_rescoring.csv"
K = 1000   # for MAP@K


# Preparing

In [4]:
# Build and save BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained(model_name_or_path)
if save_model_path is not None:
    save_tokenizer_path = "%s/tokenizer" % (save_model_path)
    tokenizer.save_pretrained(save_tokenizer_path)

# Collect mapping of all document id and text
doc_id_to_text = {}
doc_df = pd.read_csv(document_csv_path)
doc_df.fillna("<Empty Document>", inplace=True)
id_text_pair = zip(doc_df["doc_id"], doc_df["doc_text"])
for i, pair in enumerate(id_text_pair, start=1):
    doc_id, doc_text = pair
    doc_id_to_text[doc_id] = doc_text
    
    print("Progress: %d/%d\r" % (i, len(doc_df)), end='')
    
doc_df.tail()

,doc_id,doc_text
99995,LA123190-0105,CLERKS AT 13 STORES ARRESTED AFTER MINORS BUY ...
99996,LA123190-0108,LOOKING TO 1991; \n THE NEW YEAR PROMISES TRE...
99997,LA123190-0117,"LOCAL; \n GIRL, 14, DIES IN DRIVE-BY INCIDENT..."
99998,LA123190-0119,"GREECE, ISRAEL HIT BY EXODUS FROM ALBANIA \n ..."
99999,LA123190-0124,<Empty Document>


# Train 

# Split a ratio of training set as development set

In [5]:
train_df = pd.read_csv(training_csv_path)
dev_df, train_df = np.split(train_df, [int(dev_set_ratio*len(train_df))])
dev_df.reset_index(drop=True, inplace=True)
train_df.reset_index(drop=True, inplace=True)

print("train_df shape:", train_df.shape)
print("dev_df shape:", dev_df.shape)
train_df.tail()

train_df shape: (96, 5)
dev_df shape: (24, 5)


,query_id,query_text,pos_doc_ids,bm25_top1000,bm25_top1000_scores
91,641,Valdez wildlife marine life,FT911-1460 FT931-15213 FT931-16010 FT933-7162 ...,LA120989-0014 LA032390-0003 LA040889-0009 LA03...,34.16304495 32.97577181 31.31040724 30.8527172...
92,642,Tiananmen Square protesters,FBIS3-1941 FBIS3-2223 FBIS3-2224 FBIS3-26281 F...,FT922-10319 FT931-8730 FT942-5501 FBIS4-24379 ...,32.38429409 30.71831856 29.63771818 29.4676000...
93,648,family leave law,FBIS3-43072 FBIS3-61562 FBIS4-25261 FR940323-0...,FR941202-0-00181 FR941202-0-00176 FR941202-0-0...,24.51293307 23.98772391 23.42756181 23.0616218...
94,649,computer viruses,FBIS3-40468 FBIS3-42979 FBIS3-43017 FBIS4-5044...,FT944-9024 FBIS4-50440 FT921-5724 FT941-13624 ...,27.84369436 27.24267123 26.98326939 26.9108106...
95,650,tax evasion indicted,LA011689-0065 LA012589-0008 LA012889-0016 LA02...,LA040889-0060 LA053189-0041 LA092590-0146 LA06...,29.72207523 27.98961258 27.73561512 27.3372072...


# Build instances for training/development set

In [6]:
%%time
doc_id_to_token_ids = {}
def preprocess_df(df):
    ''' Preprocess DataFrame into training instances for BERT. '''
    instances = []
    
    # Parse CSV
    for i, row in df.iterrows():
        query_id, query_text, pos_doc_ids, bm25_top1000, _ = row
        pos_doc_id_list = pos_doc_ids.split()
        pos_doc_id_set = set(pos_doc_id_list)
        bm25_top1000_list = bm25_top1000.split()
        bm25_top1000_set = set(bm25_top1000_list)

        # Pair BM25 neg. with pos. samples
        labeled_pos_neg_list = []
        for pos_doc_id in pos_doc_id_list:
            neg_doc_id_set = bm25_top1000_set - pos_doc_id_set
            neg_doc_ids = random.sample(neg_doc_id_set, num_negatives)
            pos_position = random.randint(0, num_negatives)
            pos_neg_doc_ids = neg_doc_ids
            pos_neg_doc_ids.insert(pos_position, pos_doc_id)
            labeled_sample = (pos_neg_doc_ids, pos_position)
            labeled_pos_neg_list.append(labeled_sample)
            
        # Make query tokens for BERT
        query_tokens = tokenizer.tokenize(query_text)
        if len(query_tokens) > max_query_length:  # truncation
            query_tokens = query_tokens[:max_query_length]
        query_token_ids = tokenizer.convert_tokens_to_ids(query_tokens)
        query_token_ids.insert(0, tokenizer.cls_token_id)
        query_token_ids.append(tokenizer.sep_token_id)

        # Make input instances for all query/doc pairs
        for doc_ids, label in labeled_pos_neg_list:
            paired_input_ids = []
            paired_attention_mask = []
            paired_token_type_ids = []
            
            # Merge all pos/neg inputs as a single sample
            for doc_id in doc_ids:
                if doc_id in doc_id_to_token_ids:
                    doc_token_ids = doc_id_to_token_ids[doc_id]
                else:
                    doc_text = doc_id_to_text[doc_id]
                    doc_tokens = tokenizer.tokenize(doc_text)
                    doc_token_ids = tokenizer.convert_tokens_to_ids(doc_tokens)
                    doc_id_to_token_ids[doc_id] = doc_token_ids
                doc_token_ids.append(tokenizer.sep_token_id)

                # make input sequences for BERT
                input_ids = query_token_ids + doc_token_ids
                token_type_ids = [0 for token_id in query_token_ids]
                token_type_ids.extend(1 for token_id in doc_token_ids)
                if len(input_ids) > max_input_length:  # truncation
                    input_ids = input_ids[:max_input_length]
                    token_type_ids = token_type_ids[:max_input_length]
                attention_mask = [1 for token_id in input_ids]
                
                # convert and collect inputs as tensors
                input_ids = torch.LongTensor(input_ids)
                attention_mask = torch.FloatTensor(attention_mask)
                token_type_ids = torch.LongTensor(token_type_ids)
                paired_input_ids.append(input_ids)
                paired_attention_mask.append(attention_mask)
                paired_token_type_ids.append(token_type_ids)
            label = torch.LongTensor([label]).squeeze()
            
            # Pre-pad tensor pairs for efficiency
            paired_input_ids = pad_sequence(paired_input_ids, batch_first=True)
            paired_attention_mask = pad_sequence(paired_attention_mask, batch_first=True)
            paired_token_type_ids = pad_sequence(paired_token_type_ids, batch_first=True)

            # collect all inputs as a dictionary
            instance = {}
            instance['input_ids'] = paired_input_ids.T  # transpose for code efficiency
            instance['attention_mask'] = paired_attention_mask.T
            instance['token_type_ids'] = paired_token_type_ids.T
            instance['label'] = label
            instances.append(instance)

        print("Progress: %d/%d\r" % (i+1, len(df)), end='')
    print()
    return instances

train_instances = preprocess_df(train_df)
dev_instances = preprocess_df(dev_df)

print("num. train_instances: %d" % len(train_instances))
print("num. dev_instances: %d" % len(dev_instances))
print("input_ids.T shape:", train_instances[0]['input_ids'].T.shape)
train_instances[0]['input_ids'].T

Token indices sequence length is longer than the specified maximum sequence length for this model (926 > 512). Running this sequence through the model will result in indexing errors


Progress: 96/96
Progress: 24/24
num. train_instances: 7679
num. dev_instances: 1677
input_ids.T shape: torch.Size([4, 512])
CPU times: user 1min 5s, sys: 377 ms, total: 1min 5s
Wall time: 1min 5s


tensor([[  101,  3199,  3036,  ...,     0,     0,     0],
        [  101,  3199,  3036,  ..., 29153,  1998, 17800],
        [  101,  3199,  3036,  ...,     0,     0,     0],
        [  101,  3199,  3036,  ...,     0,     0,     0]])

# Build dataset and dataloader for PyTorch

In [7]:
class TrainingDataset(Dataset):
    def __init__(self, instances):
        self.instances = instances
    
    def __len__(self):
        return len(self.instances)
        
    def __getitem__(self, i):
        instance = self.instances[i]
        input_ids = instance['input_ids']
        attention_mask = instance['attention_mask']
        token_type_ids = instance['token_type_ids']
        label = instance['label']
        return input_ids, attention_mask, token_type_ids, label
    
def get_train_dataloader(instances, batch_size=2, num_workers=4):
    def collate_fn(batch):
        input_ids, attention_mask, token_type_ids, labels = zip(*batch)
        input_ids = pad_sequence(input_ids, batch_first=True).transpose(1,2).contiguous()  # re-transpose
        attention_mask = pad_sequence(attention_mask, batch_first=True).transpose(1,2).contiguous()
        token_type_ids = pad_sequence(token_type_ids, batch_first=True).transpose(1,2).contiguous()
        labels = torch.stack(labels)
        return input_ids, attention_mask, token_type_ids, labels
    
    dataset = TrainingDataset(instances)
    dataloader = DataLoader(dataset, collate_fn=collate_fn, shuffle=True, \
                            batch_size=batch_size, num_workers=num_workers)
    return dataloader

# Demo
dataloader = get_train_dataloader(train_instances)
for batch in dataloader:
    input_ids, attention_mask, token_type_ids, labels = batch
    break
    
print(input_ids.shape)
input_ids

torch.Size([2, 4, 512])


tensor([[[  101,  9732,  2943,  ...,  1996,  4762, 23725],
         [  101,  9732,  2943,  ...,     0,     0,     0],
         [  101,  9732,  2943,  ...,     0,     0,     0],
         [  101,  9732,  2943,  ...,     0,     0,     0]],

        [[  101,  2307,  3725,  ...,     0,     0,     0],
         [  101,  2307,  3725,  ...,  2095,  2857,  2683],
         [  101,  2307,  3725,  ...,  1011,  1011,  1011],
         [  101,  2307,  3725,  ...,  1012,  2007,  1037]]])

# Initialize and finetune BERT

In [8]:
model = BertForMultipleChoice.from_pretrained(model_name_or_path)
model.cuda()

optimizer = AdamW(model.parameters(), lr=learning_rate)
optimizer.zero_grad()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [9]:
def validate(model, instances):
    total_loss = 0
    model.eval()
    dataloader = get_train_dataloader(instances, batch_size=batch_size, num_workers=num_workers)
    for batch in dataloader:
        batch = (tensor.cuda() for tensor in batch)
        input_ids, attention_mask, token_type_ids, labels = batch
        
        ''' TO-DO: 
        1. Compute the cross-entropy loss (using built-in loss of BertForMultipleChoice)
          (Hint: You need to call a function of model which takes all the 4 tensors in the batch as inputs)
          
        2. Sum up the loss of all dev-set samples
          (Hint: The built-in loss is averaged, so you should multiply it with the batch size)
        '''
        with torch.no_grad():
            outputs = model(input_ids=input_ids, 
                        token_type_ids=token_type_ids, 
                        attention_mask=attention_mask, 
                        labels=labels)
            loss = outputs[0]
        total_loss += loss * batch_size
        
    avg_loss = total_loss / len(instances)
    return avg_loss

In [10]:
patience, best_dev_loss = 0, 1e10
best_state_dict = model.state_dict()

start_time = time()
dataloader = get_train_dataloader(train_instances, batch_size=batch_size, num_workers=num_workers)
for epoch in range(1, max_epochs+1):
    model.train()
    for i, batch in enumerate(dataloader, start=1):
        batch = (tensor.cuda() for tensor in batch)
        input_ids, attention_mask, token_type_ids, labels = batch
        
        # Backpropogation
        ''' TO-DO: 
        1. Compute the cross-entropy loss (using built-in loss of BertForMultipleChoice)
          (Hint: You need to call a function of model which takes all the 4 tensors in the batch as inputs)
         
        2. Perform backpropogation on the loss (i.e. compute gradients)
        3. Optimize the model.
          (Hint: These two lines of codes can be found in PyTorch tutorial)
        '''
        outputs = model(input_ids=input_ids, 
                        token_type_ids=token_type_ids, 
                        attention_mask=attention_mask, 
                        labels=labels)

        loss = outputs[0]
        
        loss.backward()
        optimizer.step()
        
        optimizer.zero_grad()
        
        # Progress bar with timer ;-)
        elapsed_time = time() - start_time
        elapsed_time = timedelta(seconds=int(elapsed_time))
        print("Epoch: %d/%d | Batch: %d/%d | loss=%.5f | %s      \r" \
              % (epoch, max_epochs, i, len(dataloader), loss, elapsed_time), end='')
        
    # Save parameters of each epoch
    if save_model_path is not None:
        save_checkpoint_path = "%s/epoch_%d" % (save_model_path, epoch)
        model.save_pretrained(save_checkpoint_path)
        
    # Get avg. loss on development set
    print("Epoch: %d/%d | Validating...                           \r" % (epoch, max_epochs), end='')
    dev_loss = validate(model, dev_instances)
    elapsed_time = time() - start_time
    elapsed_time = timedelta(seconds=int(elapsed_time))
    print("Epoch: %d/%d | dev_loss=%.5f | %s                      " \
          % (epoch, max_epochs, dev_loss, elapsed_time))
    
    # Track best checkpoint and earlystop patience
    if dev_loss < best_dev_loss:
        patience = 0
        best_dev_loss = dev_loss
        best_state_dict = deepcopy(model.state_dict())
        if save_model_path is not None:
            model.save_pretrained(save_model_path)
    else:
        patience += 1
    
    if patience > max_patience:
        print('Earlystop at epoch %d' % epoch)
        break
        
# Restore parameters with best loss on development set
model.load_state_dict(best_state_dict)

RuntimeError: CUDA out of memory. Tried to allocate 18.00 MiB (GPU 0; 7.79 GiB total capacity; 6.41 GiB already allocated; 32.00 MiB free; 6.45 GiB reserved in total by PyTorch)
Exception raised from malloc at /pytorch/c10/cuda/CUDACachingAllocator.cpp:272 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x42 (0x7fb521dab1e2 in /home/teddy/.local/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x1e64b (0x7fb52200164b in /home/teddy/.local/lib/python3.6/site-packages/torch/lib/libc10_cuda.so)
frame #2: <unknown function> + 0x1f464 (0x7fb522002464 in /home/teddy/.local/lib/python3.6/site-packages/torch/lib/libc10_cuda.so)
frame #3: <unknown function> + 0x1faa1 (0x7fb522002aa1 in /home/teddy/.local/lib/python3.6/site-packages/torch/lib/libc10_cuda.so)
frame #4: <unknown function> + 0x22f8b17 (0x7fb52450cb17 in /home/teddy/.local/lib/python3.6/site-packages/torch/lib/libtorch_cuda.so)
frame #5: <unknown function> + 0x